In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd

/lustre06/project/6023374/rozakmat/monai/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


ModuleNotFoundError: No module named 'ants'

# Define connected componnet removal

In [ ]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [ ]:
directory = Path('matt_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [4]:
len(files)

390

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [ ]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

# ANTs registration

## Upsample raw images

In [ ]:
image_path = Path('../TBI')
images = list(image_path.glob('*_*/XYZres*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

In [ ]:
for i in tqdm(range(len(images))[::-1]):
    if not os.path.exists('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name)):
        image = io.imread(images[i])
        image = np.swapaxes(image,0,1)
        image = np.swapaxes(image,1,3)
        image = resize(image,(2,507,507,252),preserve_range=True)
        zeros = np.zeros((1,507,507,252)).astype('uint16')
        image = np.append(image,zeros,axis = 0)
        image = image.astype('float16')
        io.imsave('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

In [ ]:
images[i]

## reshape numpy seg files and resave as tif

In [ ]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [ ]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[0])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

## Register images and transform masks

In [ ]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])#[0:2]
images[0]


In [ ]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('upsamp','seg',images[i])):
        if not os.path.exists(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i]))):
            fix = ants.image_read(re.sub('_0001','',images[i]))
            mov = ants.image_read(images[i])
            fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
            mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            break
            #io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
            #io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

## Save Matlab .mat file of registered images

In [ ]:
fix = ants.image_read(re.sub('_0001','',images[i]))
fix

In [ ]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])[::-1]

In [ ]:
for image in tqdm(images):
    if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

# Generate Graphs

In [ ]:
directory = Path('matt_preds')
files_seg_0001 = directory.glob('*_0001_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001

In [ ]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_0001_seg_warped.tif','_skel_warped_single.mat',file)
    #seg = io.imread(file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    io.imsave(re.sub('_seg_warped.tif','_single_skel.tif',file),skel)
    #dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.00001
    #skel_dst = skel*dst_tsfm
#
    #
    #np.save(re.sub('seg','dst_skel',file),skel_dst)
    #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel, multi=False)
    print(len(graph.edges))
    #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    
    nx.write_gpickle(graph,re.sub('_0001_seg_warped.tif','_warped.pickle',file))

# write vessel measurments to graph files

In [5]:
directory = Path('matt_preds')
files = directory.glob('*_warped.pickle')
files = sorted([x.as_posix() for x in files])
len(files)

195

In [6]:
i=0
xls = pd.ExcelFile('TBI_STIM_metalog.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(files[::-1]):
    graph = nx.read_gpickle(file)
    if len(graph.edges) < 1000:
        seg_file = re.sub('_warped.pickle','_seg_warped.tif',file)
        seg_0001_file = re.sub('_warped.pickle','_0001_seg_warped.tif',file)
        seg = io.imread(seg_file)
        seg_0001 = io.imread(seg_0001_file)
        seg_dst = distance_transform_edt(seg)
        seg_0001_dst = distance_transform_edt(seg_0001)
        for sheet_name in xls.sheet_names:
            if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)) in df[sheet_name].values:
                subj = sheet_name
                if subj in ["TBI07_3D","TBI11_3D","TBI22_3D","TBI31_3D","TBI38_3D","SHAM09_3D","SHAM12_3D","SHAM23_3D","SHAM32_3D"]:
                    gender = 'male'
                else:
                    gender = 'female'
                treatment = re.sub('SHA','SHAM',subj[0:3])
                _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('matt_preds/','',re.sub('_warped.pickle','',file))]
                if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                    wavelength = _tmp['Unnamed: 11'].iloc[0]
                    power_per = _tmp['Unnamed: 10'].iloc[0]
                    start_depth = _tmp['Unnamed: 2'].iloc[0]
                    for i in range(len(graph.edges)):
                        path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                        _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                        #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                        graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                    nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))
                    i+=1
print(i)

  5%|█▉                                         | 9/195 [02:18<53:57, 17.41s/it]/tmp/ipykernel_1184917/2457884963.py:50: RuntimeWarning:

invalid value encountered in sqrt

100%|███████████████████████████████████████| 195/195 [1:03:48<00:00, 19.63s/it]

7


In [ ]:
_tmp

In [ ]:
path[:,0]

# convert graphs to excel files

In [ ]:
directory = Path('matt_preds_graphs')
files = directory.glob('*_warped_radii.pickle')
files = sorted([x.as_posix() for x in files])
len(files)
files[0]

In [ ]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))